## Integrating rigorous desalination models into network optimization
It is often desired to design a desalination plant at a certain desalination site in the network. However, it is not clear what the optimal design and operation of the plant should be. In this notebook we show how to co-optimize the network operation and desalination design by using the integrated optimization model available in PARETO. 

First we import the necessary utilities to read the network data, the function to build the integrated model with MEE-MVR as the desalination model choice (`integrated_model_build`) and the Pyomo package.

In [ ]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from importlib import resources
from pareto.utilities.get_data import get_data
from pareto.models_extra.CM_module.cm_utils.data_parser import data_parser
from pareto.models_extra.integrate_desal.integrated_models.integrated_optimization_mvr import (
    integrated_model_build,
)
import pyomo.environ as pyo

Importing network data

In [ ]:
with resources.path(
    "pareto.models_extra.CM_module.case_studies",
    "CM_integrated_desalination_demo.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, model_type="critical_mineral")

    network_data = data_parser(df_sets, df_parameters)

Along with network data the integrated model inputs a dictionary with the name of the site at which the desalination unit needs to be built and the corresponding number of evaporator stages that need to be built at that site. For example if at site "R01_IN" we want to build an MEE-MVR unit consisting of a single evaporator stage, the dictionary will be: {"R01":1}

For more information on MEE-MVR desalination please refer to the [mee_mvr_jupyter_notebook](../desalination_jupyter_notebooks/mee_mvr_jupyter_notebook.ipynb). 

In [ ]:
desalination_dict = {"R01_IN": 1}

Now we will build the integrated model using the network data and the desalination dictionary

In [ ]:
m = integrated_model_build(network_data, treatment_dict={"R01_IN": 1})

# Solving the integrated model
Now we import IPOPT - a non-linear programming solver from the `SolverFactory` available with Pyomo to solve the integrated model.

In [ ]:
ipopt = pyo.SolverFactory("ipopt")
ipopt.options["mu_init"] = 1e-6
ipopt.options["bound_push"] = 1e-6
ipopt.solve(m, tee=True)

# Results
Now that the model is solved, we can obtain the results of our optimization problem. In this case we show how to obtain the various optimal network costs and optimal desalination CAPEX and OPEX

In [ ]:
print("=================================================================")
print("#########            Costing Variables (USD)        #############")
print("=================================================================")
print("Piping cost")
print(pyo.value(m.m_network.arc_cost))
print("=================================================================")
print("Disposal cost")
print(pyo.value(m.m_network.disp_cost))
print("=================================================================")
print("Fresh water cost")
print(pyo.value(m.m_network.fresh_cost))
print("=================================================================")
print("Storage Cost")
print(pyo.value(m.m_network.stor_cost))
print("=================================================================")
print("Storage reward")
print(pyo.value(m.m_network.stor_rev))
print("=================================================================")
print("Treatment reward")
print(pyo.value(m.m_network.treat_rev))
print("=================================================================")
print("Treatment cost - OPEX")
print(
    1000
    * pyo.value(
        sum(
            sum(m.OPEX[s, t] / 365 * m.m_network.p_dt for t in m.m_network.s_T)
            for s in m.m_network.desalination_nodes
        )
    )
)
print("Treatment cost - CAPEX")
print(
    1000
    * pyo.value(
        sum(m.CAPEX[s] for s in m.m_network.desalination_nodes)
        / 365
        * m.m_network.p_dt
        * len(m.m_network.s_T)
    )
)
print("=================================================================")

# Obtaining optimal MEE-MVR unit design
If the user wants to obtain the corresponding optimal desalination design, it can be done by using `backcalculate_mvr_design`

In [ ]:
from pareto.models_extra.integrate_desal.integrated_models.backcalculate_mvr_design import (
    back_calculate_design,
)

In [ ]:
back_calculate_design(m, treatment_dict=desalination_dict)